In [1]:
import sys, os
sys.path.append(os.path.join(os.path.dirname('__file__'), '..', 'DB_and_Azure'))
import sql_db_functions as SQLf

### get data from server

In [2]:
conn, cursor = SQLf.sql_db_functions.connect_sql()

In [3]:
query = "SELECT product_characteristics.id, Brand_id , Detail, Summary, Brand FROM product_characteristics INNER JOIN Products ON product_characteristics.Brand_id = Products.Brand_Prod_id ;"
cursor.execute(query)

# Fetch the rows
rows = cursor.fetchall()
rows

[(161,
  1,
  '    *Type: Shirt\n    *Fit: Regular fit\n    *Sleeve style: Long sleeve\n    *Neckline: Collared\n    *Material: Silk\n    *Formality: Business Casual\n    *Season: Autumn, Spring, Summer\n    *Colors: Navy (70%), White (30%)\n    *Texture: Smooth\n    *Transparency: Opaque\n    *Details and Embellishments: Buttons\n    *Shape: Fitted\n    *Length: Hip-length\n    *Collar Style: Button-down\n    *Sleeve Style: Long\n    *Patterns: Geometric\n    *Patterns placement: All-over\n    *Fluidity of fabric: High\n    *Fabric weight: Light\n    *Pocket Presence: No\n    *Pocket placement: Not applicable\n    *Pocket size: Not applicable\n    *Breathability: High\n    *Occasion Suitability: Business Casual, Formal\n    *Lapel: Not applicable',
  'The item is a Gucci silk shirt featuring an optical GG monogram print. It is designed with a regular fit and features long sleeves with buttoned cuffs. The neckline is a classic pointed collar, adding a touch of formality. The material u

In [4]:
conn, cursor = SQLf.sql_db_functions.connect_sql()

In [5]:
query = "SELECT id, Brand_id, image_link FROM product_img ;"
cursor.execute(query)

# Fetch the rows
rows_img = cursor.fetchall()
#rows_img

In [6]:
len(rows_img)

212

In [7]:
SQLf.sql_db_functions.close_connection_db(conn=conn,cursor=cursor)

### organize data

In [8]:
import pandas as pd

In [9]:
df = pd.DataFrame(rows ,columns= ['Id','prod_id','descripcion', 'Summary','Brand'])

In [10]:
df.head()

,Id,prod_id,descripcion,Summary,Brand
0,161,1,*Type: Shirt\n *Fit: Regular fit\n *...,The item is a Gucci silk shirt featuring an op...,Gucci
1,162,2,*Type: Shirt \n*Fit: Regular fit \n*Sleeve s...,The Gucci cotton Oxford GG shirt with an optic...,Gucci
2,163,3,*Type: Cropped blouse \n*Fit: Regular fit \n...,The Gucci top is a cropped blouse made of luxu...,Gucci
3,164,4,*Type: Shirt \n*Fit: Regular fit \n*Sleeve s...,The garment in question is a Gucci Camicia in ...,Gucci
4,165,5,*Type: Shirt \n*Fit: Regular fit \n*Sleeve s...,The garment in question is a Gucci cotton popl...,Gucci


In [11]:
df['descripcion'] = df['descripcion'].str.replace('\n*', ' / ')
df['descripcion'] = df['descripcion'].str.replace(r'\s+', ' ')
df['descripcion'] = df['descripcion'].str.replace('*', ' ')

df.head(5)

,Id,prod_id,descripcion,Summary,Brand
0,161,1,Type: Shirt\n Fit: Regular fit\n ...,The item is a Gucci silk shirt featuring an op...,Gucci
1,162,2,Type: Shirt / Fit: Regular fit / Sleeve s...,The Gucci cotton Oxford GG shirt with an optic...,Gucci
2,163,3,Type: Cropped blouse / Fit: Regular fit /...,The Gucci top is a cropped blouse made of luxu...,Gucci
3,164,4,Type: Shirt / Fit: Regular fit / Sleeve s...,The garment in question is a Gucci Camicia in ...,Gucci
4,165,5,Type: Shirt / Fit: Regular fit / Sleeve s...,The garment in question is a Gucci cotton popl...,Gucci


In [12]:
df['Complete_description'] = df['Summary'] + ' /' + "Brand: " + df['Brand'] + " /" + df['descripcion']
df.head()

,Id,prod_id,descripcion,Summary,Brand,Complete_description
0,161,1,Type: Shirt\n Fit: Regular fit\n ...,The item is a Gucci silk shirt featuring an op...,Gucci,The item is a Gucci silk shirt featuring an op...
1,162,2,Type: Shirt / Fit: Regular fit / Sleeve s...,The Gucci cotton Oxford GG shirt with an optic...,Gucci,The Gucci cotton Oxford GG shirt with an optic...
2,163,3,Type: Cropped blouse / Fit: Regular fit /...,The Gucci top is a cropped blouse made of luxu...,Gucci,The Gucci top is a cropped blouse made of luxu...
3,164,4,Type: Shirt / Fit: Regular fit / Sleeve s...,The garment in question is a Gucci Camicia in ...,Gucci,The garment in question is a Gucci Camicia in ...
4,165,5,Type: Shirt / Fit: Regular fit / Sleeve s...,The garment in question is a Gucci cotton popl...,Gucci,The garment in question is a Gucci cotton popl...


In [13]:
documents = df['Complete_description'].tolist()
id = df['Id'].astype('str').tolist()
metadata = [{"source": id} for id in df['Id'].astype('str').tolist()]

In [15]:
df_image = pd.DataFrame(rows_img, columns=['id','prod_id', 'Image'])

In [16]:
df_image['Image'] = df_image['Image'].apply(lambda x: x if 'https' in x else 'https:' +  x )

In [17]:
df_image.head()

,id,prod_id,Image
0,19,5,https://media.gucci.com/style/DarkGray_Center_...
1,20,5,https://media.gucci.com/style/White_Center_0_0...
2,21,6,https://media.gucci.com/style/DarkGray_Center_...
3,22,6,https://media.gucci.com/style/DarkGray_Center_...
4,23,6,https://media.gucci.com/style/DarkGray_Center_...


In [20]:
del rows, rows_img

### Get images 

In [21]:
import pandas as pd
import requests
from io import BytesIO
from PIL import Image
import base64
from fake_useragent import UserAgent

import time

In [22]:
def img_to_base64(url):
    ua = UserAgent()
    header = {'User-Agent': str(ua.chrome)}
    max_retries = 10
    retry_delay = 15  # in seconds

    for attempt in range(max_retries + 1):
        try:
            response = requests.get(url, headers=header, stream=True, timeout=5)
            response.raise_for_status()  # Raise an exception for bad status codes
            break
        except (requests.exceptions.RequestException, ValueError) as e:
            if attempt < max_retries:
                print(f"Attempt {attempt+1} failed. Retrying in {retry_delay} seconds...")
                print(url)
                time.sleep(retry_delay)
            else:
                print(f"All {max_retries} attempts failed. Giving up.")
                return 'Retry'

    img = Image.open(BytesIO(response.content))

    buffered = BytesIO()
    if url.lower().endswith('.png'):
        img.save(buffered, format="PNG")
        return "data:image/png;base64, " + str(base64.b64encode(buffered.getvalue()))[2:-1]
    else:
        img.save(buffered, format="JPEG")
        return "data:image/jpeg;base64, " + str(base64.b64encode(buffered.getvalue()))[2:-1]



In [23]:
df_image.sort_values(by='prod_id',ascending=True,inplace=True)
df_image.reset_index(inplace=True,drop=True)
df_image.head()

,id,prod_id,Image
0,1,1,https://media.gucci.com/style/DarkGray_Center_...
1,4,1,https://media.gucci.com/style/White_Center_0_0...
2,2,1,https://media.gucci.com/style/DarkGray_Center_...
3,3,1,https://media.gucci.com/style/DarkGray_Center_...
4,5,2,https://media.gucci.com/style/DarkGray_Center_...


In [ ]:
data_embedded = []


for i in range(0, len(df_image)-1):

    current_image = df_image.Image.iloc[i]

    v = img_to_base64(current_image)

    data_embedded = data_embedded + [v]

    if i % 10 == 0: print(i)



In [25]:
import base64
from io import BytesIO
from PIL import Image


def base64_to_image(base64_string):
    # Remove the data URI prefix if present
    if "data:image" in base64_string:
        base64_string = base64_string.split(",")[1]
    # Decode the Base64 string into bytes
    image_bytes = base64.b64decode(base64_string)
    return image_bytes

def create_image_from_bytes(image_bytes):
    # Create a BytesIO object to handle the image data
    image_stream = BytesIO(image_bytes)

    # Open the image using Pillow (PIL)
    image = Image.open(image_stream)
    return image

def resize_image(image, new_width):
    # Open the image

    # Calculate the new height while maintaining the aspect ratio
    original_width, original_height = image.size
    aspect_ratio = original_height / original_width
    new_height = int(new_width * aspect_ratio)

    # Resize the image
    resized_image = image.resize((new_width, new_height))

    return resized_image

In [26]:
data_embedded[0]

'data:image/jpeg;base64, ...'

In [27]:
if len(data_embedded) != len(df_image):
    data_embedded = data_embedded + ['Retry']

In [31]:
i = 10

In [32]:
df_image.iloc[i].Image

'https://media.gucci.com/style/White_Center_0_0_730x490/1712821642/788987_ZAQRX_6190_001_100_0000_Light-Top-corto-in-seta-con-stampa-Morsetto-e-righe-Gucci.jpg'

In [ ]:
resize_image(
    create_image_from_bytes(image_bytes=base64_to_image( 
        data_embedded[i]  
    )),
400)

### Testing embeddings

In [34]:
df_image['base64'] = data_embedded

In [35]:
df_image[df_image.prod_id == 60]

,id,prod_id,Image,base64
164,168,60,https://www.prada.com/content/dam/pradabkg_pro...,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
165,167,60,https://www.prada.com/content/dam/pradabkg_pro...,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."


In [36]:
for i in df_image[df_image.prod_id == 60].index:
    print(df_image.iloc[i].Image)

https://www.prada.com/content/dam/pradabkg_products/P/P43/P439H/15GGF0E18/P439H_15GG_F0E18_S_OOO_MDD.jpg/_jcr_content/renditions/cq5dam.web.hebebed.1800.1800.jpg
https://www.prada.com/content/dam/pradabkg_products/P/P43/P439H/15GGF0E18/P439H_15GG_F0E18_S_OOO_MDF.jpg/_jcr_content/renditions/cq5dam.web.hebebed.1800.1800.jpg


In [ ]:
i = df_image[df_image.prod_id == 60].index[0]

resize_image(
    create_image_from_bytes(image_bytes=base64_to_image( 
        df_image.iloc[i].base64
    )),
400)

In [ ]:
i = df_image[df_image.prod_id == 60].index[1]

resize_image(
    create_image_from_bytes(image_bytes=base64_to_image( 
        df_image.iloc[i].base64
    )),
400)

In [39]:
def base64_to_image(base64_string):
    # Remove the data URI prefix if present
    if "data:image" in base64_string:
        base64_string = base64_string.split(",")[1]

    # Decode the Base64 string into bytes
    image_bytes = base64.b64decode(base64_string)
    return image_bytes

In [40]:
image_bytes = base64.b64decode(df_image.iloc[0].base64[23:])

In [41]:
def create_image_from_bytes(image_bytes):
    # Create a BytesIO object to handle the image data
    image_stream = BytesIO(image_bytes)

    # Open the image using Pillow (PIL)
    image = Image.open(image_stream)
    return image

In [42]:
df_image.head()

,id,prod_id,Image,base64
0,1,1,https://media.gucci.com/style/DarkGray_Center_...,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
1,4,1,https://media.gucci.com/style/White_Center_0_0...,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
2,2,1,https://media.gucci.com/style/DarkGray_Center_...,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
3,3,1,https://media.gucci.com/style/DarkGray_Center_...,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
4,5,2,https://media.gucci.com/style/DarkGray_Center_...,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."


In [43]:
for i in df_image[df_image['base64'] == 'Retry'].index:
    print(i)

22
23
41
48
211


In [ ]:
data_embedded = []


for i in df_image[df_image['base64'] == 'Retry'].index:

    current_image = df_image.Image.iloc[i]

    v = img_to_base64(current_image)

    data_embedded = data_embedded + [v]

    print(i)


In [50]:
i = 22
#i = 23
#i = 41
#i = 48
#i = 211

In [51]:
df_image.iloc[i].Image

'https://media.gucci.com/style/DarkGray_Center_0_0_980x980/1712822418/791840_ZAQ1Q_6253_001_100_0000_Light-Camicia-in-seta-con-stampa-Morsetto.jpg'

In [ ]:
resize_image(
    create_image_from_bytes(image_bytes=base64_to_image( 
        data_embedded[0]
    )),
400)

In [53]:
n = 0 
for i in df_image[df_image['base64'] == 'Retry'].index:
    print(f'i={i} -- n={n}')
    print(df_image.iloc[i].base64 == data_embedded[n])

    n+=1

i=22 -- n=0
False
i=23 -- n=1
False
i=41 -- n=2
False
i=48 -- n=3
False
i=211 -- n=4
False


In [54]:
n = 0 
for i in df_image[df_image['base64'] == 'Retry'].index:
    print(f'i={i} -- n={n}')
    df_image.loc[i,'base64'] = data_embedded[n]

    n+=1

i=22 -- n=0
i=23 -- n=1
i=41 -- n=2
i=48 -- n=3
i=211 -- n=4


In [55]:
n = 0 
for i in df_image[df_image['base64'] == 'Retry'].index:
    print(f'i={i} -- n={n}')
    print(df_image.iloc[i].base64 == data_embedded[n])

    n+=1

In [56]:
df_image.tail()

,id,prod_id,Image,base64
207,209,81,https://www.prada.com/content/dam/pradabkg_pro...,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
208,212,82,https://www.prada.com/content/dam/pradabkg_pro...,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
209,211,82,https://www.prada.com/content/dam/pradabkg_pro...,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
210,213,83,https://www.prada.com/content/dam/pradabkg_pro...,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
211,214,83,https://www.prada.com/content/dam/pradabkg_pro...,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."


### Save embeddings in the DB

In [57]:
import psycopg2

In [58]:
conn, cursor = SQLf.sql_db_functions.connect_sql()

In [59]:
for index, row in df_image.iterrows():
    base64 = row['base64']
    i = row['id']
    
    # Update the base64 column in the product_img table
    cursor.execute("UPDATE product_img SET base64 = %s WHERE id = %s", (base64, i))

In [ ]:
query = "SELECT * FROM product_img ;"
cursor.execute(query)

# Fetch the rows
rows_img = cursor.fetchall()
rows_img

In [61]:
conn.commit()